# Data cleaning

This notebook processes the raw data exported, following EDA and feedback with data owner.

In [ ]:
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 100)

%matplotlib inline

In [ ]:
# Load data
original_data_df = pd.read_parquet("../../data/original-data.parquet")
original_data_df.shape

## Convert datetimes

In [ ]:
datetimes_df = original_data_df.copy()
datetime_cols = [
    "DISCHARGE_DATE_HOSPITAL_PROVIDER_SPELL",
    "EXPECTED_DISCHARGE_DATE",
    "FIRST_START_DATE_TIME_WARD_STAY",
    "START_DATE_TIME_HOSPITAL_PROVIDER_SPELL",
]

for col in datetime_cols:
    datetimes_df[col] = pd.to_datetime(datetimes_df[col], format="%Y-%m-%d %H:%M:%S.%f")

## Order rows

Original data is ~unordered, order by START_DATE_TIME_HOSPITAL_PROVIDER_SPELL

In [ ]:
datetimes_df.sort_values(by="START_DATE_TIME_HOSPITAL_PROVIDER_SPELL", inplace=True)
datetimes_df.reset_index(drop=True, inplace=True)

## Drop early rows missing data

As explored during EDA, there is a section of ~110k rows missing data due to the introduction of a new clinical system.

Once ordered by admission, this section appears at the start. Remove these rows rather than impute.

In [ ]:
# Check missing rows appear at start
datetimes_df.wait_minutes.isnull().map({True: 1, False: 0}).plot();

In [ ]:
# Remove null values in first 110,000 values, and append the remaining values into a new dataframe
modern_data_df = datetimes_df[datetimes_df.index < 110000].dropna(
    axis=0, subset=["wait_minutes"]
)
modern_data_df = pd.concat(
    [modern_data_df, datetimes_df[datetimes_df.index >= 110000]]
).reset_index(drop=True)
# Visualise remaining rows
modern_data_df.wait_minutes.isnull().map({True: 1, False: 0}).plot()
modern_data_df.shape

## Drop empty/redundant/agreed columns

As agreed with data SME

In [ ]:
cleaned_cols_df = (
    modern_data_df.drop(
        # Drop empty columns
        columns=[
            "DISCHARGE_READY_DATE",
            "cds_unique_identifier",
            "healthcare_resource_group_code",
            "presenting_complaint_code",
            "ae_patient_group_code",
            "ae_patient_group",
        ]
    )
    .drop(
        # Drop redundant columns
        columns=[
            "Frailty Proxy",
            "all_breach_reason_codes",
            "ae_attendance_category_code",
            "all_diagnosis_codes",
            "all_investigation_codes",
            "all_local_investigation_codes",
            "all_local_treatment_codes",
            "all_treatment_codes",
            "PATIENT_CLASSIFICATION",
            "PATIENT_GENDER_CURRENT",
            "SOURCE_OF_ADMISSION_HOSPITAL_PROVIDER_SPELL",
            "TREATMENT_FUNCTION_CODE_AT_ADMISSION",
            "MAIN_SPECIALTY_CODE_AT_ADMISSION",
            "ae_initial_assessment_triage_category_code",
            "ae_initial_assessment_triage_category",
            "major_minor",
            "manchester_triage_category",
            "FIRST_START_DATE_TIME_WARD_STAY",
            "FIRST_REGULAR_DAY_OR_NIGHT_ADMISSION_DESCRIPTION",
            "wait",
            "attendance_type",
            "initial_wait",
            "arrival_day_of_week",
            "arrival_month_name",
        ]
    )
    .drop(
        # Drop identifier columns
        columns=[
            "LOCAL_PATIENT_IDENTIFIER",
            "previous_30_day_hospital_provider_spell_number",
            "ED_attendance_episode_number",
            "unique_internal_ED_admission_number",
            "unique_internal_IP_admission_number",
        ]
    )
    .drop(
        # Drop low cardinality columns
        columns=[
            "FIRST_WARD_STAY_IDENTIFIER",
            "LENGTH_OF_STAY_IN_MINUTES",
            "START_DATE_HOSPITAL_PROVIDER_SPELL",
            "EXPECTED_DISCHARGE_DATE_TIME",
        ]
    )
    .drop(
        # Drop less useful columns
        columns=["wait_minutes", "initial_wait_minutes"]
    )
)
cleaned_cols_df.shape

## Assign nan values 

* SME agrees that NaN = N for stroke_ward_stay
* SME agrees that None = N for IS_MAJOR

In [ ]:
cleaned_cols_df.stroke_ward_stay.value_counts()

In [ ]:
# fill stroke_ward_stay
imputed_df = cleaned_cols_df.copy()
imputed_df.stroke_ward_stay.fillna(value="N", inplace=True)
imputed_df.stroke_ward_stay.value_counts()

In [ ]:
imputed_df.IS_major.value_counts()

In [ ]:
# fill IS_major
imputed_df.IS_major.fillna(value="N", inplace=True)
imputed_df.IS_major.value_counts()

## Drop sparse rows

In [ ]:
# check for null values across dataset
imputed_df.isnull().sum()

In [ ]:
# some columns have < 1000 null values, and a larger subset have ~68000 missing. Drop these
removed_sparse_rows_df = imputed_df.dropna(
    subset=[
        "ADMISSION_METHOD_HOSPITAL_PROVIDER_SPELL_DESCRIPTION",
        "EXPECTED_DISCHARGE_DATE",
        "MAIN_SPECIALTY_CODE_AT_ADMISSION_DESCRIPTION",
        "POST_CODE_AT_ADMISSION_DATE_DISTRICT",
        "IMD county decile",
        "all_diagnoses",
    ]
)

## Remove duplicates

In [ ]:
no_duplicate_rows_df = removed_sparse_rows_df.drop_duplicates()

## Homogenise binary fields

Many fields are encoding as Y/N or similar, convert these into binary fields

In [ ]:
binary_fields_df = no_duplicate_rows_df.copy()
binary_fields_df.stroke_ward_stay = binary_fields_df.stroke_ward_stay.apply(
    lambda x: 0 if x == "N" else 1
)
binary_fields_df.IS_care_home_on_admission = (
    binary_fields_df.IS_care_home_on_admission.apply(lambda x: 0 if x == "N" else 1)
)
binary_fields_df.IS_care_home_on_discharge = (
    binary_fields_df.IS_care_home_on_discharge.apply(lambda x: 0 if x == "N" else 1)
)
binary_fields_df.IS_major = binary_fields_df.IS_major.map({"Y": 1, "N": 0})
# create new fields
binary_fields_df["IS_illness_not_injury"] = binary_fields_df["Illness Injury Flag"].map(
    {"Illness": 1, "Injury": 0}
)
binary_fields_df["IS_elective"] = binary_fields_df.elective_or_non_elective.map(
    {"Non-elective admission": 0, "Elective admission": 1}
)
# drop old
binary_fields_df.drop(
    columns=["Illness Injury Flag", "elective_or_non_elective"], inplace=True
)

In [ ]:
# check new binary fields
for field in [
    "stroke_ward_stay",
    "IS_care_home_on_admission",
    "IS_care_home_on_discharge",
    "IS_illness_not_injury",
    "IS_elective",
    "IS_major",
]:
    print(
        f"{field} has {binary_fields_df[field].isnull().sum()} null values and values:"
    )
    print(binary_fields_df[field].value_counts())

## Check genders

In [ ]:
# there are only 13 "not specified" gender, all others are M/F
binary_fields_df.PATIENT_GENDER_CURRENT_DESCRIPTION.value_counts()

In [ ]:
# drop "not specified" values
genders_df = binary_fields_df.drop(
    labels=binary_fields_df[
        binary_fields_df.PATIENT_GENDER_CURRENT_DESCRIPTION == "Not specified"
    ].index
)
genders_df.shape

## Final data checks

In [ ]:
# check null values
# there are still some columns with majority (~400k) values null; these can be encoding in a null field during modelling e.g. ae_arrival_mode
genders_df.isnull().sum()

In [ ]:
# plot null values
sns.set(rc={"figure.figsize": (15, 8)})
sns.heatmap(genders_df.isnull(), cbar=False);

## Export cleaned data

In [ ]:
# Export data (outside git tree)
genders_df.to_parquet("../../data/clean-data.parquet")

In [ ]:
# Export cols/descriptions for Excel/Google Sheets import (inside git tree)
genders_df.dtypes.to_csv("../data/cols.csv")
genders_df.describe().to_csv("../data/describe.csv")